In [1]:
import polars as pl

In [2]:
dataset = pl.read_csv('data/healthcare_dataset.csv')
display(dataset.head())

Name,Age,Gender,Blood Type,Medical Condition,Date of Admission,Doctor,Hospital,Insurance Provider,Billing Amount,Room Number,Admission Type,Discharge Date,Medication,Test Results
str,i64,str,str,str,str,str,str,str,f64,i64,str,str,str,str
"""Bobby JacksOn""",30,"""Male""","""B-""","""Cancer""","""2024-01-31""","""Matthew Smith""","""Sons and Miller""","""Blue Cross""",18856.281306,328,"""Urgent""","""2024-02-02""","""Paracetamol""","""Normal"""
"""LesLie TErRy""",62,"""Male""","""A+""","""Obesity""","""2019-08-20""","""Samantha Davies""","""Kim Inc""","""Medicare""",33643.327287,265,"""Emergency""","""2019-08-26""","""Ibuprofen""","""Inconclusive"""
"""DaNnY sMitH""",76,"""Female""","""A-""","""Obesity""","""2022-09-22""","""Tiffany Mitchell""","""Cook PLC""","""Aetna""",27955.096079,205,"""Emergency""","""2022-10-07""","""Aspirin""","""Normal"""
"""andrEw waTtS""",28,"""Female""","""O+""","""Diabetes""","""2020-11-18""","""Kevin Wells""","""Hernandez Rogers and Vang,""","""Medicare""",37909.78241,450,"""Elective""","""2020-12-18""","""Ibuprofen""","""Abnormal"""
"""adrIENNE bEll""",43,"""Female""","""AB+""","""Cancer""","""2022-09-19""","""Kathleen Hanna""","""White-White""","""Aetna""",14238.317814,458,"""Urgent""","""2022-10-09""","""Penicillin""","""Abnormal"""


In [3]:
# Transform the Name column with only initial letters uppercase and the rest lowercase
dataset = dataset.with_columns(pl.col('Name').str.to_titlecase().alias('Name'))

# I round the Billing Amount column to just two decimal places
dataset = dataset.with_columns(pl.col('Billing Amount').round(2).alias('Billing Amount'))

In [4]:
# Observe columns, data types, null values
print(dataset.describe())

shape: (9, 16)
┌────────────┬────────────┬───────────┬────────┬───┬───────────┬───────────┬───────────┬───────────┐
│ statistic  ┆ Name       ┆ Age       ┆ Gender ┆ … ┆ Admission ┆ Discharge ┆ Medicatio ┆ Test      │
│ ---        ┆ ---        ┆ ---       ┆ ---    ┆   ┆ Type      ┆ Date      ┆ n         ┆ Results   │
│ str        ┆ str        ┆ f64       ┆ str    ┆   ┆ ---       ┆ ---       ┆ ---       ┆ ---       │
│            ┆            ┆           ┆        ┆   ┆ str       ┆ str       ┆ str       ┆ str       │
╞════════════╪════════════╪═══════════╪════════╪═══╪═══════════╪═══════════╪═══════════╪═══════════╡
│ count      ┆ 55500      ┆ 55500.0   ┆ 55500  ┆ … ┆ 55500     ┆ 55500     ┆ 55500     ┆ 55500     │
│ null_count ┆ 0          ┆ 0.0       ┆ 0      ┆ … ┆ 0         ┆ 0         ┆ 0         ┆ 0         │
│ mean       ┆ null       ┆ 51.539459 ┆ null   ┆ … ┆ null      ┆ null      ┆ null      ┆ null      │
│ std        ┆ null       ┆ 19.602454 ┆ null   ┆ … ┆ null      ┆ null      ┆

In [5]:
# Transform the 'Date of Admission' column from object to date

dataset = dataset.with_columns([
    pl.col('Date of Admission').str.to_date('%Y-%m-%d').alias('Date of Admission'),
    pl.col('Discharge Date').str.to_date('%Y-%m-%d').alias('Discharge Date')
])

In [6]:
# Checking for duplicate rows
dataset_duplicati = dataset.is_duplicated().sum()
print('Rihe duplicate:', dataset_duplicati)

# Removing duplicate rows
dataset = dataset.unique()

Rihe duplicate: 1068


Data Cleanup and Transformation
During the initial analysis of the dataset, we found that the Name column contained incorrectly spelled patient names; we therefore corrected this.

We rounded the Billing Amount column to two decimal places.

We then analyzed the dataset to observe its composition, highlighting the total number of rows (55,500) and columns (15), and checking for null values ​​as well as the data type of each column.

I changed the data type of the Date of Admission and Discharge Date columns from object to datetime64.

Demographic Analysis
    Distribution of patients by age, sex, and blood type
    Mean and median age by type of admission or medical condition
    Map of patients by hospital, to understand where certain profiles are concentrated

In [7]:
# Distribution of patients by age, sex, and blood type

dataset = dataset.with_columns(
    pl.when(pl.col('Age') < 20).then(pl.lit('teenager'))
    .when(pl.col('Age') < 40).then(pl.lit('young adults'))
    .when(pl.col('Age') < 60).then(pl.lit('adult'))
    .otherwise(pl.lit('old'))
    .alias('Age group')
)

ageGroup = (dataset.group_by('Age group')
            .agg(pl.len().alias('patients'))
            .with_columns((pl.col('patients') * 100 / len(dataset)).round(2).alias('%'))
            .sort('patients', descending=True))
display(ageGroup)

gender = dataset.group_by('Gender').agg(pl.len().alias('patients')).sort('patients', descending=True)
display(gender)

bloodType = dataset.group_by('Blood Type').agg(pl.len().alias('patients')).sort('patients', descending=True)
display(bloodType)

Age group,patients,%
str,u32,f64
"""old""",20948,38.11
"""adult""",16308,29.67
"""young adults""",16033,29.17
"""teenager""",1677,3.05


Gender,patients
str,u32
"""Male""",27496
"""Female""",27470


Blood Type,patients
str,u32
"""A-""",6898
"""A+""",6896
"""B+""",6885
"""AB+""",6882
"""AB-""",6874
"""B-""",6872
"""O+""",6855
"""O-""",6804


In [8]:
# Mean and median age by type of admission or medical condition

medicalCondition = (dataset.group_by('Medical Condition')
                    .agg([pl.col('Age').mean().round(2).alias('mean'),
                          pl.col('Age').median().round(2).alias('median')]))
display(medicalCondition)

admissionType = (dataset.group_by('Admission Type')
                 .agg([pl.col('Age').mean().round(2).alias('mean'),
                       pl.col('Age').median().round(2).alias('median')]))
display(admissionType)

Medical Condition,mean,median
str,f64,f64
"""Cancer""",51.54,52.0
"""Hypertension""",51.72,52.0
"""Obesity""",51.23,51.0
"""Arthritis""",51.54,51.0
"""Asthma""",51.6,52.0
"""Diabetes""",51.58,52.0


Admission Type,mean,median
str,f64,f64
"""Elective""",51.44,52.0
"""Emergency""",51.54,52.0
"""Urgent""",51.63,52.0


In [9]:
# Map of patients by hospital, to understand where certain profiles are concentrated

patientsMap = dataset.group_by('Hospital').agg(pl.len().alias('patients')).sort('patients', descending=True)
display(patientsMap.head(20))

Hospital,patients
str,u32
"""LLC Smith""",44
"""Ltd Smith""",39
"""Johnson PLC""",37
"""Smith Ltd""",37
"""Smith PLC""",36
…,…
"""Inc Brown""",27
"""Inc Smith""",27
"""Group Johnson""",26


Clinical Analysis
    Frequency of medical conditions (which conditions are most common)
    Relationship between medical conditions and gender, age, and blood type
    Most prescribed medications for each diagnosis
    Average length of hospital stay by medical condition and age

In [10]:
# Frequency of medical conditions (which conditions are most common)
medicalCondition = dataset.group_by('Medical Condition').agg(pl.len().alias('patients')).sort('patients', descending=True)
display(medicalCondition)

Medical Condition,patients
str,u32
"""Arthritis""",9218
"""Diabetes""",9216
"""Hypertension""",9151
"""Obesity""",9146
"""Cancer""",9140
"""Asthma""",9095


In [11]:
# Relationship between medical conditions and gender, age, and blood type
def contingency_table(column1, column2):
    contingency = (dataset.group_by([column1, column2])
                   .agg(pl.len().alias('count'))
                   .with_columns((pl.col('count') / pl.col('count').sum().over(column1) * 100).round(4).alias('percentage'))
                   .pivot(values='percentage', index=column1, on=column2))
    display(contingency)

contingency_table('Medical Condition', 'Gender')
contingency_table('Medical Condition', 'Age group')
contingency_table('Medical Condition', 'Blood Type')

Medical Condition,Male,Female
str,f64,f64
"""Hypertension""",50.071,49.929
"""Diabetes""",49.9891,50.0109
"""Asthma""",50.4013,49.5987
"""Obesity""",50.0,50.0
"""Cancer""",50.0438,49.9562
"""Arthritis""",49.642,50.358


Medical Condition,old,adult,teenager,young adults
str,f64,f64,f64,f64
"""Hypertension""",38.422,29.658,3.027,28.893
"""Arthritis""",37.7848,29.8655,3.2111,29.1386
"""Diabetes""",38.2812,29.9588,2.9297,28.8303
"""Cancer""",38.337,29.1575,3.0853,29.4201
"""Asthma""",38.5926,29.0269,3.0346,29.3458
"""Obesity""",37.2513,30.3411,3.0177,29.3899


Medical Condition,A+,O-,B+,A-,AB+,O+,B-,AB-
str,f64,f64,f64,f64,f64,f64,f64,f64
"""Asthma""",12.3584,12.5234,12.1385,12.7763,12.8972,12.7763,12.1825,12.3474
"""Hypertension""",12.2719,12.403,11.8785,12.9603,13.157,12.5123,12.709,12.108
"""Arthritis""",12.0091,12.3129,12.9204,12.4105,12.1935,12.8661,12.4973,12.7902
"""Cancer""",12.8118,12.5164,12.9759,12.2976,12.035,12.0131,12.3742,12.9759
"""Diabetes""",13.0317,12.0009,12.8038,12.5217,12.6411,12.3915,12.3264,12.283
"""Obesity""",12.7925,12.5191,12.4317,12.3333,12.2021,12.2677,12.9237,12.5301


In [12]:
#  Most prescribed medications for each diagnosis
count_Medication = (dataset.group_by(['Medical Condition', 'Medication'])
                    .agg(pl.len().alias('count'))
                    .sort(['Medical Condition', 'count'], descending=[False, True]))

max_Medication = (count_Medication.group_by('Medical Condition')
                  .agg([pl.col('Medication').first(), pl.col('count').first()]))

display(max_Medication)

Medical Condition,Medication,count
str,str,u32
"""Arthritis""","""Aspirin""",1901
"""Asthma""","""Paracetamol""",1870
"""Cancer""","""Lipitor""",1904
"""Diabetes""","""Lipitor""",1875
"""Hypertension""","""Ibuprofen""",1874
"""Obesity""","""Penicillin""",1872


In [13]:
# Average length of hospital stay by medical condition and age

dataset = dataset.with_columns(
    (pl.col('Discharge Date') - pl.col('Date of Admission')).dt.total_days().alias('Days In Hospital')
)

def calculate_TimeInHospital(column1): 
    avg_TimeHospital = (dataset.group_by(column1)
                        .agg(pl.col('Days In Hospital').mean().round(2).alias('Days In Hospital')))
    display(avg_TimeHospital)

calculate_TimeInHospital('Medical Condition')
calculate_TimeInHospital('Age group')

Medical Condition,Days In Hospital
str,f64
"""Obesity""",15.45
"""Asthma""",15.68
"""Arthritis""",15.5
"""Cancer""",15.5
"""Hypertension""",15.44
"""Diabetes""",15.43


Age group,Days In Hospital
str,f64
"""young adults""",15.38
"""teenager""",15.45
"""old""",15.58
"""adult""",15.53


Economic Analysis
Percentage of costs covered by each insurance provider.
Average billing amounts by insurance and admission type.
Average cost per recovery day.

In [14]:
# Percentage of costs covered by each insurance provider.
total_billing = dataset.select(pl.col('Billing Amount').sum()).item()
insurance_costs = (dataset.group_by('Insurance Provider')
                   .agg(pl.col('Billing Amount').sum().round(2).alias('Billing Amount'))
                   .with_columns((pl.col('Billing Amount') / total_billing * 100).round(2).alias('%Insurance Costs')))
display(insurance_costs)

Insurance Provider,Billing Amount,%Insurance Costs
str,f64,f64
"""UnitedHealthcare""",2.7992e8,19.94
"""Blue Cross""",2.8041e8,19.97
"""Cigna""",2.8433e8,20.25
"""Aetna""",2.7650e8,19.69
"""Medicare""",2.8291e8,20.15


In [15]:
# Average billing amounts by insurance and admission type.
def calculate_avg_biling_amount(column1):
    avg_biling_amout = (dataset.group_by(column1)
                        .agg(pl.col('Billing Amount').mean().round(2).alias('Avg Billing Amount')))
    display(avg_biling_amout)


calculate_avg_biling_amount('Admission Type')
calculate_avg_biling_amount('Insurance Provider')

Admission Type,Avg Billing Amount
str,f64
"""Elective""",25612.14
"""Urgent""",25514.53
"""Emergency""",25505.33


Insurance Provider,Avg Billing Amount
str,f64
"""Medicare""",25628.32
"""Cigna""",25526.0
"""Aetna""",25549.69
"""Blue Cross""",25603.46
"""UnitedHealthcare""",25414.51


In [16]:
# Average cost per recovery day.
avg_cost = round((dataset.select(pl.col('Billing Amount').sum()).item() / dataset.select(pl.col('Days In Hospital').sum()).item()), 2)
print("Average cost per day of hospitalization:", avg_cost)

Average cost per day of hospitalization: 1648.1


Temporal Analysis¶
Monthly hospitalization rates (identify seasonality)
Monthly discharge rates

In [17]:
# Monthly hospitalization rates (identify seasonality)

dataset = dataset.with_columns(pl.col('Date of Admission').dt.month().alias('Moth Admission'))
month_admission_hospitalizations = (dataset.group_by('Moth Admission')
                                    .agg(pl.len().alias('Patients'))
                                    .sort('Patients', descending=True))
display(month_admission_hospitalizations)

Moth Admission,Patients
i8,u32
8,4785
7,4765
1,4655
6,4650
3,4622
…,…
5,4555
9,4508
11,4508


In [18]:
 # Monthly discharge rates

dataset = dataset.with_columns(pl.col('Discharge Date').dt.month().alias('Discharge Month'))
discharge_month_hospitalizations = (dataset.group_by('Discharge Month')
                                    .agg(pl.len().alias('Patients'))
                                    .sort('Patients', descending=True))
display(discharge_month_hospitalizations)

Discharge Month,Patients
i8,u32
7,4746
8,4721
9,4677
5,4654
1,4630
…,…
3,4572
6,4526
11,4484


Operational and Medical Analysis¶
Number of patients per doctor
Abnormal test rate by admission type

In [19]:
# Number of patients per doctor
patients_per_doctor = (dataset.group_by('Doctor')
                       .agg(pl.len().alias('Patients'))
                       .sort('Patients', descending=True))
display(patients_per_doctor.head(15))

Doctor,Patients
str,u32
"""Michael Smith""",27
"""John Smith""",22
"""Robert Smith""",21
"""Michael Johnson""",20
"""James Smith""",20
…,…
"""Christopher Smith""",17
"""Michael Brown""",16
"""William Johnson""",15


In [20]:
# Abnormal test rate by admission type

abnormal_test_rate = (dataset.filter(pl.col('Test Results') == 'Abnormal')
                      .group_by('Admission Type')
                      .agg(pl.len().alias('Test Abnormal'))
                      .sort('Test Abnormal', descending=True))
display(abnormal_test_rate)

Admission Type,Test Abnormal
str,u32
"""Elective""",6232
"""Urgent""",6167
"""Emergency""",6038
